In [1]:
import pandas as pd
import re

In [2]:
netflix_data = pd.read_csv('../data/raw/netflix_branded_titles_with_release_year.csv')

In [3]:
#split combined_title into title and season
netflix_data.rename(columns={'title_desc':'combined_title'}, inplace=True)
netflix_data[['title', 'season']] = netflix_data['combined_title'].str.split(': Season ', expand=True)
netflix_data['title'] = netflix_data['title'].str.lower()
netflix_data['is_tv_show'] = netflix_data['season'].notnull()
netflix_data['release_year'] = netflix_data['release_year'].astype('int')

In [4]:
#read imdb movie data
imdb_movie_data = pd.read_csv('../data/processed/imdb_movies.csv')

In [5]:
#read matched data
matched_data = pd.read_csv('../data/raw/matched_wIMDB.tsv', sep='\t')[['tconst', 'title', 'release_year', 'season']]

In [15]:
imdb_clean_movie = imdb_movie_data[['tconst', 'title', 'startYear']].rename(columns={'startYear':'release_year'})

In [16]:
imdb_clean2 = pd.concat([imdb_clean_movie, matched_data], axis=0)

In [45]:
movie_merge = netflix_data[~netflix_data['is_tv_show']].merge(imdb_clean2, on=['title', 'release_year'], how='left', indicator=True)

In [65]:
good_batch_1 = movie_merge[(movie_merge['_merge']=='both') & (movie_merge['tconst'].notnull())].groupby('title', as_index=False).filter(lambda x: len(x) == 1)[['tconst', 'title', 'release_year']]
good_batch_1.to_csv('../data/processed/good_batch_1.csv', index=False)
#824 unique matches on title alone
#Drops to 818 if match on year

In [68]:
manual_match_1 = good_batch_1.groupby('title', as_index=False).filter(lambda x: len(x) > 1)[['tconst', 'title', 'release_year']].drop_duplicates()
manual_match_1.to_csv('../data/processed/manual_match_1.csv', index=False)

In [69]:
fails = movie_merge[movie_merge['_merge']=='left_only']

In [6]:
imdb_all = pd.read_csv('../data/processed/imdb_all.csv').rename(columns={'startYear':'release_year'})

In [74]:
movie_merge2 = fails[['title', 'release_year']].merge(imdb_all, on=['title', 'release_year'], how='left', indicator=True)

In [78]:
good_batch_2 = movie_merge2[(movie_merge2['_merge']=='both') & (movie_merge2['tconst'].notnull())].groupby('title', as_index=False).filter(lambda x: len(x) == 1)[['tconst', 'title', 'release_year']]
good_batch_2.to_csv('../data/processed/good_batch_2.csv', index=False)

In [79]:
manual_match_2 = movie_merge2[(movie_merge2['_merge']=='both') & (movie_merge2['tconst'].notnull())].groupby('title', as_index=False).filter(lambda x: len(x) > 1)[['tconst', 'title', 'release_year']].drop_duplicates()
manual_match_2.to_csv('../data/processed/manual_match_2.csv', index=False)

In [96]:
manual_match_3 = movie_merge2[movie_merge2['_merge']=='left_only'][['title', 'release_year']]
manual_match_3.to_csv('../data/processed/manual_match_3.csv', index=False)

In [102]:
print(f"Correctly Matched: {netflix_data[~netflix_data['is_tv_show']].shape[0] - good_batch_1.shape[0] - good_batch_2.shape[0]}\n")
print(f"Incorrectly Matched Duplicates Movies: {len(manual_match_1['title'].unique())}")
print(f"Incorrectly Matched Duplicate Non-movies: {len(manual_match_2['title'].unique())}")
print(f"Incorrectly Matched No Match: {len(manual_match_3['title'].unique())}")

#Remaining 119
#23 in manual_match_1
#6 in manual_match_2
#90 in manual_match_3


Correctly Matched: 119

Incorrectly Matched Duplicates Movies: 23
Incorrectly Matched Duplicate Non-movies: 6
Incorrectly Matched No Match: 90


## Repeat for TV Shows

In [192]:
#read imdb tv data
imdb_tv_data = pd.read_csv('../data/processed/imdb_tv.csv')[['tconst', 'title', 'startYear']].rename(columns={'startYear':'release_year'})

1021 Shows

In [210]:
#netflix_data[netflix_data['is_tv_show']].shape

(1121, 7)

In [234]:
tv_merge1 = netflix_data[netflix_data['is_tv_show']].merge(imdb_tv_data, on=['title'], how='left', indicator=True)

Somehow we already lost 4. I know 3 are because of duplicate title + season, but not sure where the last 1 went.

Left with 1117

In [235]:
#tv_merge1[['title', 'season']].drop_duplicates().shape

In [236]:
good_batch_tv_1 = tv_merge1[(tv_merge1['_merge']=='both') & (tv_merge1['tconst'].notnull())].groupby(['title', 'season'], as_index=False).filter(lambda x: len(x) == 1)[['tconst', 'title', 'season', 'release_year_x']].rename(columns={'release_year_x':'release_year'})
manual_match_tv_1 = tv_merge1[(tv_merge1['_merge']=='both') & (tv_merge1['tconst'].notnull())].groupby(['title', 'season'], as_index=False).filter(lambda x: len(x) > 1)[['tconst', 'title', 'season', 'release_year_x']].rename(columns={'release_year_x':'release_year'})

In [245]:
print(f"Matched 1:1: {good_batch_tv_1.shape[0]}")
print(f"Matched 1:M: {manual_match_tv_1[['title', 'season']].drop_duplicates().shape[0]}")

Matched 1:1: 878
Matched 1:M: 154


Immediately 878 match 1:1, 154 match 1:M, 85 don't match. Everything OK up to here

In [247]:
#chip away at some of the manual_tv_1 matches using zach match
#already_matched_titles = manual_match_tv_1[manual_match_tv_1['title'].isin(matched_data['title'])]['title'].unique()
#manual_match_tv_2 = manual_match_tv_1[~manual_match_tv_1['title'].isin(already_matched_titles)]
#manual_match_tv_2.to_csv('../data/processed/manual_match_tv_1.csv', index=False)

In [218]:
#add the already matched to the good batch
#already_matched_df = matched_data[matched_data['title'].isin(already_matched_titles)][['tconst', 'title', 'season', 'release_year']]
#good_batch_tv_1 = pd.concat([good_batch_tv_1, already_matched_df], axis=0)
#good_batch_tv_1.to_csv('../data/processed/good_batch_tv_1.csv', index=False)

Of the 154 1:M, 9 are already matched via Zach's list. 145 remain

In [253]:
#9 matches
good_batch_tv_2 = manual_match_tv_1[['title', 'season', 'release_year']].merge(matched_data, on=['title', 'season', 'release_year'], how='inner').drop_duplicates()
good_batch_tv_2.to_csv('../data/processed/good_batch_tv_2.csv', index=False)

Of the 145 remaining, 83 match on season 1 release year. 62 remaining

In [291]:
#Clean more manual matches using release year with season 1
good_batch_tv_3 = manual_match_tv_1[['title', 'season', 'release_year']].drop_duplicates().merge(imdb_tv_data, on=['title', 'release_year'], how='inner')
good_batch_tv_3 = good_batch_tv_3.groupby(['title', 'season'], as_index=False).filter(lambda x: len(x) == 1)[['tconst', 'title', 'season', 'release_year']]
good_batch_tv_3.to_csv('../data/processed/good_batch_tv_3.csv', index=False)
print(good_batch_tv_3.shape)

(83, 4)


62 remaining manual matches with potential ttvalues

In [298]:
checker = pd.concat([good_batch_tv_2, good_batch_tv_3], axis=0).merge(manual_match_tv_1, on=['title', 'season'], how='outer', indicator=True)
remaining_manual_1 = checker[checker['_merge']=='right_only'][['title', 'season', 'release_year_y', 'tconst_y']].rename(columns={'release_year_y':'release_year', 'tconst_y':'tconst'})

remaining_manual_1.to_csv('../data/processed/manual_match_tv_1_updated.csv', index=False)

In [221]:
# 1121 tv shows
# 887 matched 
# 145 multiple ttcodes


Move on to the 85 non-matches

In [246]:
fails_tv = tv_merge1[tv_merge1['_merge']=='left_only']
print(fails_tv.shape)

(85, 10)


In [300]:
#Now try matching failed matches using zach's data
tv_merge2 = fails_tv[['title', 'season', 'release_year_x']].merge(matched_data, on=['title', 'season'], how='left', indicator=True)
good_batch_tv_4 = tv_merge2[(tv_merge2['_merge']=='both') & (tv_merge2['tconst'].notnull())].groupby(['title', 'season'], as_index=False).filter(lambda x: len(x) == 1)[['tconst', 'title', 'season', 'release_year']].rename(columns={'release_year_x':'release_year'})
manual_match_tv_2 = tv_merge2[(tv_merge2['_merge']=='left_only')][['title', 'season', 'release_year_x']].rename(columns={'release_year_x':'release_year'})

66 Match with Zach's list. 19 remaining

In [302]:
good_batch_tv_4.to_csv('../data/processed/good_batch_tv_2.csv', index=False)
#66 good matches

19 Remaining need a manual match

In [304]:
#manual_match_tv_2.shape
manual_match_tv_2.to_csv('../data/processed/manual_match_tv_2.csv', index=False)